# Dataset Creation

Package for creating synthetic PIV images:

In [ ]:
import synpivimage as spi 
spi.__version__

For the notebook we retrieve the dataset directory from the dot-env file:

In [ ]:
from dotenv import load_dotenv
import os
import numpy as np

load_dotenv('generation.env')
dataset_dir = os.getenv('dataset_dir')
dataset_dir

## 1. Define an inital configuration
The initial configuration shall be the default one of the synpivimage package. From there, we adjust the parameters.<br>
Note that the image width/height is chosen to be odd. This is recommended when using the auto-correlation feature later on.

In [ ]:
cfg = spi.DEFAULT_CFG

cfg['bit_depth'] = 16
cfg['nx'] = 19
cfg['ny'] = 19
cfg['laser_max_intensity'] = 1000
image_size = cfg['nx']*cfg['ny']
cfg

## 2. Defining parameters to be varied

Create tuples with the configuration name and the parameters. The parameter values can be a number only, a list or a numpy array (Obviously passing a number is equal to not specify the variation tuple at all).<br>
Last we specify how many images per combinations will be generated with `n_per_combination`

In [ ]:
particle_number_range = ('particle_number', np.arange(1, image_size/2, 1).astype(int))
particle_mean_size_range = ('particle_size_mean', 3)
lasershape_range = ('laser_shape_factor', 10)
baseline_range = ('noise_baseline', 100)
dark_noise_range = ('dark_noise', 10)

n_per_combination = 1

## 3. Generate a list of all config combinations:
Before we can write the data to disk, we generate a `ConfigManager`. This class is initalize by passing a list of all configuration dictionaries that define each image generation. Thus we need 101 configuration dictionaries. Using `build_ConfgManger()` we retrieve the `ConfigManger`-instance by passing the inital config dictionary and the variation tuples from above (so we don't need to deal with building the man config dictionaries ourselves):

In [ ]:
CFGs = spi.build_ConfigManager(cfg, [particle_number_range,
                                     particle_mean_size_range,
                                     lasershape_range,
                                     baseline_range,
                                     dark_noise_range],
                              per_combination = n_per_combination,
                              shuffle=True)
CFGs

## 4. Write all data to HDF5 files:
With a large amount of parameters the image generation may produce arrays that will blow your RAM. Therefore define the maximal number of images to be created before written to the file. This parameter is called `n_split`. If using `None`, all data is written into a single file. If you pass any integer number, then that number of files will be created. Filenames will look like `<dataset_dir>/ds_000001.hdf` and so on.

In [ ]:
%%time
hdf_filename = CFGs.to_hdf(dataset_dir, nproc=4, n_split=None, overwrite=True)  # n_split=None will not take a split, thus a single HDF5 file will be created

In [ ]:
import h5py
import matplotlib.pyplot as plt

with h5py.File(hdf_filename[0], 'r') as h5:
    plt.imshow(h5['image'][41, :, :], cmap='gray')